In [1]:
import re
import tensorflow as tf
import numpy as np
import time
import os
import nltk
import collections
import random
import time
import pickle

In [2]:
def clearstring(string):
    string = re.sub('[^A-Za-z\- ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    return [y.lower() for y in string if len(y) > 3 and y.find('nbsp') < 0]

def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    global data
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            for word in data[:span]:
                buffer.append(word)
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [3]:
with open('books/gameofthrones.txt', 'r',encoding = "ISO-8859-1") as fopen:
    vocabulary = clearstring(' '.join(nltk.word_tokenize(fopen.read())))
print('example 10 words:',vocabulary[:10])
print('size corpus:',len(vocabulary))
vocabulary_size = len(list(set(vocabulary)))
print('size of unique words:',vocabulary_size)
dimension = 128
skip_window = 1
num_skips = 2
batch_size = 64
location = os.getcwd()

example 10 words: ['should', 'start', 'back', 'gared', 'urged', 'woods', 'began', 'grow', 'dark', 'around']
size corpus: 580815
size of unique words: 20055


In [4]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 1], ('that', 8719), ('with', 7724), ('said', 5678), ('they', 5445)]
Sample data [77, 1203, 19, 2398, 1201, 851, 263, 924, 160, 83] ['should', 'start', 'back', 'gared', 'urged', 'woods', 'began', 'grow', 'dark', 'around']


In [5]:
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [6]:
graph = tf.Graph()

with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, dimension], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, dimension],
                                                      stddev=1.0 / np.sqrt(dimension)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                             biases=nce_biases,
                                             labels=train_labels,
                                             inputs=embed,
                                             num_sampled=batch_size / 2,
                                             num_classes=vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
        init = tf.global_variables_initializer()

num_steps = 200000

In [7]:
with tf.Session(graph=graph) as session:
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  147.49130249
Nearest to some: gallantly, rebuffed, bitter, enjoy, pretenders, indignant, nonsense, potion,
Nearest to could: unhappily, upbringing, from, vale, buggers, falconers, alert, maynard,
Nearest to were: shining, clumsily, boomed, patrolled, cutpurses, knuckled, masterless, separate,
Nearest to night: poleaxed, dozed, shoulder, selected, gurn, tourneys, ankle, browns,
Nearest to look: scandal, inordinately, thunderstorm, grotesquerie, hefted, spoke, cartwheels, nesting,
Nearest to will: ilyns, crabapple, char, duur, loose, sealord, parmens, piecrust,
Nearest to tyrion: anymore, bricks, phantoms, spent, starved, unpleasant, tighter, chests,
Nearest to said: regular, slits, whole, twisty, eater, bride-to-be, halberd, shrewdly,
Nearest to make: musk, prowlers, mischance, sharp-faced, pox-scarred, lodger, erryk, surcoats,
Nearest to made: tastiest, sizable, twinkling, tracts, swirl, vargo, stubbornness, heated,
Nearest to father: breeze, aler

Average loss at step  52000 :  4.13768442154
Average loss at step  54000 :  4.11437428236
Average loss at step  56000 :  4.08218713796
Average loss at step  58000 :  3.98948419738
Average loss at step  60000 :  3.95093563157
Nearest to some: this, extravagance, things, gallantly, there, rhaegar, gladly, whites,
Nearest to could: would, will, must, seemed, should, might, wanted, made,
Nearest to were: theyre, been, dwindled, their, payment, fletching, dragonflies, foothills,
Nearest to night: place, passed, time, exulting, when, bouncing, sleeves, slime,
Nearest to look: looked, borrowed, that, thinks, undone, talk, veteran, silver-and-crimson,
Nearest to will: would, must, shall, should, might, could, does, youll,
Nearest to tyrion: catelyn, arya, dany, sansa, jaime, bran, cersei, davos,
Nearest to said: asked, thought, smiled, told, spoke, agreed, replied, called,
Nearest to make: find, give, tell, symon, keep, carpenters, take, hold,
Nearest to made: could, would, stubbornness, gave,

Average loss at step  112000 :  3.75879953361
Average loss at step  114000 :  3.70675306654
Average loss at step  116000 :  3.73180647242
Average loss at step  118000 :  3.75898490322
Average loss at step  120000 :  3.6875409106
Nearest to some: extravagance, gallantly, date, things, lock, this, gallerys, retain,
Nearest to could: would, will, shielded, couldnt, wanted, should, made, youll,
Nearest to were: theyre, dwindled, foothills, sparr, touching, dragonflies, rend, payment,
Nearest to night: vhaghar, gladness, place, mending, case, morning, slime, ominously,
Nearest to look: looked, confused, demonstrating, borrowed, behave, freakish, talk, laughed,
Nearest to will: would, shall, should, must, might, youll, wont, could,
Nearest to tyrion: catelyn, arya, dany, davos, cersei, jaime, littlefinger, varys,
Nearest to said: asked, told, replied, smiled, thought, agreed, answered, spoke,
Nearest to make: give, carpenters, symon, making, made, take, squirms, find,
Nearest to made: could,

Average loss at step  172000 :  3.60981705344
Average loss at step  174000 :  3.58879098535
Average loss at step  176000 :  3.57137350523
Average loss at step  178000 :  3.61254517579
Average loss at step  180000 :  3.59111745131
Nearest to some: extravagance, date, this, things, gallantly, conjurers, every, corsair,
Nearest to could: couldnt, would, will, cant, shielded, made, wont, scroll,
Nearest to were: theyre, dwindled, rend, werent, touching, appearance, payment, witless,
Nearest to night: gladness, afternoon, swam, tender-hearted, mending, slime, ominously, case,
Nearest to look: looked, throw, demonstrating, borrowed, leave, pennies, fast-fading, appear,
Nearest to will: would, shall, must, should, might, youll, wont, dont,
Nearest to tyrion: catelyn, jaime, arya, cersei, davos, dany, varys, littlefinger,
Nearest to said: asked, replied, told, answered, says, smiled, agreed, nodded,
Nearest to make: offshore, symon, take, give, happen, making, carpenters, surround,
Nearest to 

In [8]:
with open('gameofthrone-list.p', 'wb') as fopen:
    pickle.dump(list(reverse_dictionary.values()), fopen)

In [9]:
with open('gameofthrone-vector.p', 'wb') as fopen:
    pickle.dump(final_embeddings, fopen)